# GREEN AI Projetct - DIA6
# Energy Optimization in New York's Buildings Using Artificial Intelligence
Gabriel SULTAN,
Yasmine ZEROUAL,
Lalith Adithya CHANUMOLU

In [16]:
!mkdir -p data/GREENprojet/ashrae

In [17]:
!ls data/GREENprojet/ashrae

building_metadata.csv	   test.csv	  weather_test.feather
building_metadata.feather  test.feather   weather_train.feather
sample_submission.csv	   train.csv
sample_submission.feather  train.feather


In [18]:
import pandas as pd

# Weather train
df = pd.read_feather("data/GREENprojet/ashrae/weather_train.feather")
df.to_csv("weather_train.csv", index=False)

# Weather test
df = pd.read_feather("data/GREENprojet/ashrae/weather_test.feather")
df.to_csv("weather_test.csv", index=False)

# Train
df = pd.read_feather("data/GREENprojet/ashrae/train.feather")
df.to_csv("train.csv", index=False)

# Test
df = pd.read_feather("data/GREENprojet/ashrae/test.feather")
df.to_csv("test.csv", index=False)

# Building metadata
df = pd.read_feather("data/GREENprojet/ashrae/building_metadata.feather")
df.to_csv("building_metadata.csv", index=False)

# Sample submission
df = pd.read_feather("data/GREENprojet/ashrae/sample_submission.feather")
df.to_csv("sample_submission.csv", index=False)


In [20]:
import pandas as pd

# Define data types for each column to optimize memory usage
dtypes = {
    'site_id': 'int8',
    'air_temperature': 'float32',
    'cloud_coverage': 'float32',
    'dew_temperature': 'float32',
    'precip_depth_1_hr': 'float32',
    'sea_level_pressure': 'float32',
    'wind_direction': 'float32',
    'wind_speed': 'float32'
}

weather_file = '../../data/GREENprojet/ashrae/weather_train.csv'
parse_dates = ['timestamp']

# 'infer_datetime_format' is deprecated and can be safely omitted
weather_df = pd.read_csv(
    weather_file,
    dtype=dtypes,
    parse_dates=parse_dates,
    low_memory=True
)

weather_df.head()


FileNotFoundError: [Errno 2] No such file or directory: '../../data/GREENprojet/ashrae/weather_train.csv'

In [ ]:
# Charger le fichier train.csv avec optimisation de la mémoire
# Ce fichier contient les données de consommation énergétique pour l'entraînement

# Définir les types de données pour optimiser la mémoire
train_dtypes = {
    'building_id': 'int16',
    'meter': 'int8',
    'meter_reading': 'float32'
}

train_file = '../../data/GREENprojet/ashrae/train.csv'

print("⏳ Chargement du fichier train.csv (cela peut prendre quelques minutes)...")

# Charger le fichier avec types optimisés
train_df = pd.read_csv(
    train_file,
    dtype=train_dtypes,
    parse_dates=['timestamp'],
    low_memory=False
)

print(f"✅ Chargement terminé !")
print(f"📊 Dimensions: {train_df.shape[0]:,} lignes × {train_df.shape[1]} colonnes")
print(f"💾 Mémoire utilisée: {train_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

train_df.head()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Display basic information about the weather dataset
print("Weather Data Info:")
print(weather_df.info())
print("\nWeather Data Description:")
print(weather_df.describe())
print("\nWeather Data Missing Values:")
print(weather_df.isnull().sum())

# Plotting distributions of major weather features
features_to_plot = [
    'air_temperature', 'dew_temperature',
    'cloud_coverage', 'precip_depth_1_hr',
    'sea_level_pressure', 'wind_direction', 'wind_speed'
]

plt.figure(figsize=(20, 14))
for i, feature in enumerate(features_to_plot):
    plt.subplot(3, 3, i+1)
    sns.histplot(weather_df[feature].dropna(), kde=True, bins=50)
    plt.title(f'Distribution of {feature}')
plt.tight_layout()
plt.show()

# Time series trends for air_temperature
plt.figure(figsize=(16,6))
for site in weather_df['site_id'].unique()[:3]:  # Plotting for a few sites
    site_df = weather_df[weather_df['site_id'] == site]
    plt.plot(site_df['timestamp'], site_df['air_temperature'], label=f'Site {site}', alpha=0.7)
plt.title("Air Temperature Trend for Sample Sites")
plt.xlabel("Timestamp")
plt.ylabel("Air Temperature (°C)")
plt.legend()
plt.show()

# Heatmap of missing values in a random sample
plt.figure(figsize=(10,6))
sns.heatmap(weather_df.sample(1000).isnull(), cbar=False, cmap='viridis')
plt.title("Missing Values Heatmap (Sample of 1000 Rows)")
plt.show()

print("missing values in the weather data")
print(weather_df.isnull().sum())
print("\nrange of air temperatures recorded")
print(weather_df['air_temperature'].describe())




In [ ]:
# Exploring the distribution of the target variable: meter_reading

# Check if train_df is loaded
if 'train_df' in globals() and train_df is not None and 'meter_reading' in train_df.columns:
    print("Summary Statistics for meter_reading:")
    print(train_df['meter_reading'].describe())

    plt.figure(figsize=(12,6))
    sns.histplot(train_df['meter_reading'], bins=100, kde=True)
    plt.title('Distribution of Meter Reading')
    plt.xlabel('Meter Reading')
    plt.ylabel('Frequency')
    plt.show()

    # Checking distribution after log(1 + x) transform (to address skewness)
    plt.figure(figsize=(12,6))
    sns.histplot(np.log1p(train_df['meter_reading']), bins=100, kde=True, color='orange')
    plt.title('Distribution of log(1 + Meter Reading)')
    plt.xlabel('log(1 + Meter Reading)')
    plt.ylabel('Frequency')
    plt.show()

    zero_count = (train_df['meter_reading'] == 0).sum()
    print(f"Number of zero meter readings: {zero_count}")
else:
    print("train_df is not loaded or does not contain 'meter_reading' column.")



Observations :

Meter reading values are highly skewed, and it is impossible to visualize raw meter values.
We can apply log transformation to make the distribution normal

A right-skewed distribution can be attributed to a large number of zero or missing meter readings. It is impossible for a meter to record zero values, thereby suggesting missing meter values. We can drop off these values before training the model.

In [ ]:
# Remove rows where meter_reading is zero or missing, and create a cleaned DataFrame for modeling
if 'train_df' in globals() and train_df is not None and 'meter_reading' in train_df.columns:
    # Drop rows with NaN meter_reading first (should be rare)
    cleaned_train_df = train_df.dropna(subset=['meter_reading'])
    # Then drop rows where meter_reading is exactly zero
    cleaned_train_df = cleaned_train_df[cleaned_train_df['meter_reading'] != 0]
    print(f"Original training set shape: {train_df.shape}")
    print(f"Shape after removing zero/missing meter_reading rows: {cleaned_train_df.shape}")
else:
    print("train_df is not loaded or does not contain 'meter_reading' column.")


In [ ]:
# Time series analysis of meter readings for different meters

if 'cleaned_train_df' in globals() and cleaned_train_df is not None:
    # Ensure 'timestamp' is datetime
    cleaned_train_df['timestamp'] = pd.to_datetime(cleaned_train_df['timestamp'])

    # Aggregate (sum) meter readings by meter type and timestamp (daily)
    meter_types = {0: 'Electricity', 1: 'Chilled Water', 2: 'Steam', 3: 'Hot Water'}
    cleaned_train_df['meter_label'] = cleaned_train_df['meter'].map(meter_types)

    # Daily total meter readings for each meter type
    daily_meter = (
        cleaned_train_df
        .groupby(['timestamp', 'meter_label'])['meter_reading']
        .sum()
        .reset_index()
    )
    # Pivot for easier plotting
    daily_meter_pivot = daily_meter.pivot(index='timestamp', columns='meter_label', values='meter_reading')

    # Plot timeseries for each meter type (total meter reading per day)
    plt.figure(figsize=(20, 8))
    for meter in meter_types.values():
        if meter in daily_meter_pivot.columns:
            plt.plot(daily_meter_pivot.index, daily_meter_pivot[meter], label=meter)
    plt.title('Total Daily Meter Readings by Meter Type (Train Set)')
    plt.xlabel('Date')
    plt.ylabel('Total Meter Reading')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Mean and total energy consumption by meter type
    meter_stats = cleaned_train_df.groupby('meter_label')['meter_reading'].agg(['mean', 'sum']).sort_values('mean', ascending=False)
    print("Mean and Total Meter Readings by Meter Type:")
    print(meter_stats)
else:
    print("cleaned_train_df is not loaded or not defined.")


In [ ]:
# Charger les métadonnées des bâtiments
building_metadata_file = '../../data/GREENprojet/ashrae/building_metadata.csv'

# Définir les types de données pour optimiser la mémoire (éviter float16 car pandas ne le supporte pas en lecture)
building_dtypes = {
    'site_id': 'int8',
    'building_id': 'int16',
    'square_feet': 'int32',
    'year_built': 'float32',      # Remplacer 'float16' par 'float32'
    'floor_count': 'float32'      # Remplacer 'float16' par 'float32'
}

print("⏳ Chargement des métadonnées des bâtiments...")

building_metadata_df = pd.read_csv(
    building_metadata_file,
    dtype=building_dtypes,
    low_memory=False
)

print(f"✅ Chargement terminé !")
print(f"📊 Dimensions: {building_metadata_df.shape[0]:,} bâtiments × {building_metadata_df.shape[1]} caractéristiques")
print(f"💾 Mémoire utilisée: {building_metadata_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Afficher les premières lignes
print("\n🏢 Aperçu des métadonnées des bâtiments :")
display(building_metadata_df.head(10))


In [ ]:
# Analyse exploratoire des métadonnées des bâtiments

print("="*80)
print("INFORMATIONS GÉNÉRALES SUR LES BÂTIMENTS")
print("="*80)

print("\n📊 Informations sur les colonnes :")
print(building_metadata_df.info())

print("\n📈 Statistiques descriptives :")
print(building_metadata_df.describe())

print("\n❓ Valeurs manquantes :")
missing_values = building_metadata_df.isnull().sum()
missing_percent = (missing_values / len(building_metadata_df)) * 100
missing_df = pd.DataFrame({
    'Valeurs manquantes': missing_values,
    'Pourcentage (%)': missing_percent
})
print(missing_df[missing_df['Valeurs manquantes'] > 0])

print(f"\n🏢 Nombre de bâtiments uniques : {building_metadata_df['building_id'].nunique()}")
print(f"📍 Nombre de sites uniques : {building_metadata_df['site_id'].nunique()}")
print(f"🏗️ Nombre d'usages primaires uniques : {building_metadata_df['primary_use'].nunique()}")

print("\n🏗️ Répartition des bâtiments par site :")
print(building_metadata_df['site_id'].value_counts().sort_index())


In [ ]:
# Visualisation des distributions des caractéristiques des bâtiments

fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# 1. Distribution de la superficie (square_feet)
axes[0, 0].hist(building_metadata_df['square_feet'].dropna(), bins=50, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Distribution de la Superficie (square_feet)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Superficie (pieds carrés)')
axes[0, 0].set_ylabel('Fréquence')
axes[0, 0].ticklabel_format(style='plain', axis='x')

# 2. Distribution de l'année de construction (year_built)
axes[0, 1].hist(building_metadata_df['year_built'].dropna(), bins=50, color='lightcoral', edgecolor='black')
axes[0, 1].set_title('Distribution de l\'Année de Construction', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Année de construction')
axes[0, 1].set_ylabel('Fréquence')

# 3. Distribution du nombre d'étages (floor_count)
axes[0, 2].hist(building_metadata_df['floor_count'].dropna(), bins=50, color='lightgreen', edgecolor='black')
axes[0, 2].set_title('Distribution du Nombre d\'Étages', fontsize=14, fontweight='bold')
axes[0, 2].set_xlabel('Nombre d\'étages')
axes[0, 2].set_ylabel('Fréquence')

# 4. Boxplot de la superficie
axes[1, 0].boxplot(building_metadata_df['square_feet'].dropna(), vert=True)
axes[1, 0].set_title('Boxplot de la Superficie', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Superficie (pieds carrés)')

# 5. Boxplot de l'année de construction
axes[1, 1].boxplot(building_metadata_df['year_built'].dropna(), vert=True)
axes[1, 1].set_title('Boxplot de l\'Année de Construction', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('Année')

# 6. Boxplot du nombre d'étages
axes[1, 2].boxplot(building_metadata_df['floor_count'].dropna(), vert=True)
axes[1, 2].set_title('Boxplot du Nombre d\'Étages', fontsize=14, fontweight='bold')
axes[1, 2].set_ylabel('Nombre d\'étages')

plt.tight_layout()
plt.show()

# Statistiques sur les bâtiments anciens et modernes
print("\n🏛️ ANALYSE TEMPORELLE DES BÂTIMENTS")
print("="*60)
oldest_year = building_metadata_df['year_built'].min()
newest_year = building_metadata_df['year_built'].max()
print(f"Bâtiment le plus ancien : {oldest_year}")
print(f"Bâtiment le plus récent : {newest_year}")
print(f"Étendue : {int(newest_year - oldest_year)} ans")

# Statistiques sur la taille des bâtiments
print("\n🏢 ANALYSE DE LA TAILLE DES BÂTIMENTS")
print("="*60)
print(f"Superficie minimale : {building_metadata_df['square_feet'].min():,.0f} pieds carrés")
print(f"Superficie maximale : {building_metadata_df['square_feet'].max():,.0f} pieds carrés")
print(f"Superficie médiane : {building_metadata_df['square_feet'].median():,.0f} pieds carrés")

# Statistiques sur les étages
print("\n🏗️ ANALYSE DES ÉTAGES")
print("="*60)
print(f"Nombre minimal d'étages : {building_metadata_df['floor_count'].min():.0f}")
print(f"Nombre maximal d'étages : {building_metadata_df['floor_count'].max():.0f}")
print(f"Nombre médian d'étages : {building_metadata_df['floor_count'].median():.0f}")


### Usage Primaire vs. Relevés de Compteur

Nous allons maintenant voir si la caractéristique `primary_use` peut être sélectionnée pour prédire les relevés de compteur.


In [ ]:
# Analyse de l'usage primaire (primary_use)

print("="*80)
print("ANALYSE DE L'USAGE PRIMAIRE DES BÂTIMENTS")
print("="*80)

# Compter le nombre de bâtiments par usage primaire
primary_use_counts = building_metadata_df['primary_use'].value_counts()
print("\n📊 Nombre de bâtiments par usage primaire :")
print(primary_use_counts)

# Visualisation : Distribution des usages primaires
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# Graphique à barres
primary_use_counts.plot(kind='bar', ax=axes[0], color='teal', edgecolor='black')
axes[0].set_title('Nombre de Bâtiments par Usage Primaire', fontsize=16, fontweight='bold')
axes[0].set_xlabel('Usage Primaire', fontsize=12)
axes[0].set_ylabel('Nombre de Bâtiments', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Graphique circulaire (pie chart) pour les top 10
top_10_uses = primary_use_counts.head(10)
colors = plt.cm.tab20c(range(len(top_10_uses)))
axes[1].pie(top_10_uses, labels=top_10_uses.index, autopct='%1.1f%%', startangle=90, colors=colors)
axes[1].set_title('Top 10 des Usages Primaires (% de bâtiments)', fontsize=16, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n✅ Une majorité de bâtiments ont un usage primaire 'Education', suivi par 'Office'.")


In [ ]:
# Fusionner les métadonnées des bâtiments avec les données de consommation énergétique

print("⏳ Fusion des métadonnées des bâtiments avec les relevés de compteur...")

# Fusionner cleaned_train_df avec building_metadata_df sur building_id
merged_df = cleaned_train_df.merge(
    building_metadata_df,
    on='building_id',
    how='left'
)

print(f"✅ Fusion terminée !")
print(f"📊 Dimensions du DataFrame fusionné : {merged_df.shape[0]:,} lignes × {merged_df.shape[1]} colonnes")
print(f"💾 Mémoire utilisée: {merged_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Vérifier les premières lignes
print("\n🔍 Aperçu des données fusionnées :")
merged_df.head(10)


In [ ]:
# Analyse de la consommation énergétique par usage primaire

print("="*80)
print("CONSOMMATION ÉNERGÉTIQUE PAR USAGE PRIMAIRE")
print("="*80)

# Calculer les statistiques par usage primaire
primary_use_stats = merged_df.groupby('primary_use')['meter_reading'].agg([
    ('Nombre de relevés', 'count'),
    ('Consommation moyenne', 'mean'),
    ('Consommation totale', 'sum'),
    ('Consommation médiane', 'median'),
    ('Écart-type', 'std')
]).sort_values('Consommation moyenne', ascending=False)

print("\n📊 Statistiques de consommation énergétique par usage primaire :")
print(primary_use_stats)

# Visualisation : Consommation moyenne par usage primaire
fig, axes = plt.subplots(1, 2, figsize=(22, 7))

# Graphique 1 : Consommation moyenne
primary_use_stats['Consommation moyenne'].sort_values(ascending=True).plot(
    kind='barh',
    ax=axes[0],
    color='steelblue',
    edgecolor='black'
)
axes[0].set_title('Consommation Énergétique Moyenne par Usage Primaire', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Consommation moyenne (kWh)', fontsize=12)
axes[0].set_ylabel('Usage Primaire', fontsize=12)
axes[0].grid(axis='x', alpha=0.3)

# Graphique 2 : Consommation totale
primary_use_stats['Consommation totale'].sort_values(ascending=True).plot(
    kind='barh',
    ax=axes[1],
    color='coral',
    edgecolor='black'
)
axes[1].set_title('Consommation Énergétique Totale par Usage Primaire', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Consommation totale (kWh)', fontsize=12)
axes[1].set_ylabel('Usage Primaire', fontsize=12)
axes[1].ticklabel_format(style='scientific', axis='x', scilimits=(0,0))
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ La consommation énergétique moyenne est principalement influencée par 'Education' et 'Services'.")


De la meme maniere nous pourrions réaliser une analyse en comparant^la consomation à l'age du batiment, le nombre d'étage, la superficie...

## Analyse de l'Impact de Site ID et Building ID sur la Consommation Énergétique

Nous allons maintenant analyser si les caractéristiques `site_id` et `building_id` affectent la consommation énergétique afin de **détecter les valeurs aberrantes (outliers)**.


### Analyse de la Consommation par Site (Site ID)

Les relevés de compteur ont été enregistrés sur **16 sites différents**. Analysons les différences de consommation énergétique entre ces sites.


In [ ]:
# Analyse de la consommation énergétique par site

print("="*80)
print("ANALYSE DE LA CONSOMMATION ÉNERGÉTIQUE PAR SITE")
print("="*80)

# Calculer les statistiques par site
site_stats = merged_df.groupby('site_id')['meter_reading'].agg([
    ('Nombre de relevés', 'count'),
    ('Consommation moyenne', 'mean'),
    ('Consommation médiane', 'median'),
    ('Consommation totale', 'sum'),
    ('Écart-type', 'std'),
    ('Min', 'min'),
    ('Max', 'max')
]).sort_values('Consommation moyenne', ascending=False)

print("\n📊 Statistiques de consommation par site :")
print(site_stats)

# Calculer le coefficient de variation (CV) pour identifier la variabilité
site_stats['Coefficient de variation (%)'] = (site_stats['Écart-type'] / site_stats['Consommation moyenne']) * 100

print("\n📈 Sites avec la plus forte variabilité (Coefficient de Variation) :")
print(site_stats[['Consommation moyenne', 'Coefficient de variation (%)']].sort_values('Coefficient de variation (%)', ascending=False))

# Nombre de bâtiments par site
buildings_per_site = building_metadata_df.groupby('site_id').size().reset_index(name='Nombre de bâtiments')
print("\n🏢 Nombre de bâtiments par site :")
print(buildings_per_site)


In [ ]:
# Visualisation de la consommation par site

fig, axes = plt.subplots(2, 2, figsize=(20, 14))

# 1. Consommation moyenne par site
site_stats['Consommation moyenne'].plot(
    kind='bar',
    ax=axes[0, 0],
    color='steelblue',
    edgecolor='black'
)
axes[0, 0].set_title('Consommation Énergétique Moyenne par Site', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Site ID', fontsize=12)
axes[0, 0].set_ylabel('Consommation Moyenne (kWh)', fontsize=12)
axes[0, 0].grid(axis='y', alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=0)

# 2. Consommation totale par site
site_stats['Consommation totale'].plot(
    kind='bar',
    ax=axes[0, 1],
    color='coral',
    edgecolor='black'
)
axes[0, 1].set_title('Consommation Énergétique Totale par Site', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Site ID', fontsize=12)
axes[0, 1].set_ylabel('Consommation Totale (kWh)', fontsize=12)
axes[0, 1].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))
axes[0, 1].grid(axis='y', alpha=0.3)
axes[0, 1].tick_params(axis='x', rotation=0)

# 3. Boxplot de la consommation par site (échelle log)
sample_sites = merged_df.sample(n=min(100000, len(merged_df)), random_state=42)
sample_sites.boxplot(
    column='meter_reading',
    by='site_id',
    ax=axes[1, 0],
    patch_artist=True
)
axes[1, 0].set_title('Distribution de la Consommation par Site (échelle log)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Site ID', fontsize=12)
axes[1, 0].set_ylabel('Consommation (kWh)', fontsize=12)
axes[1, 0].set_yscale('log')
plt.suptitle('')

# 4. Coefficient de variation par site
site_stats['Coefficient de variation (%)'].sort_values().plot(
    kind='barh',
    ax=axes[1, 1],
    color='lightgreen',
    edgecolor='black'
)
axes[1, 1].set_title('Variabilité de la Consommation par Site (Coefficient de Variation)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Coefficient de Variation (%)', fontsize=12)
axes[1, 1].set_ylabel('Site ID', fontsize=12)
axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()


### Détection des Outliers par Site

Utilisons des méthodes statistiques pour identifier les sites avec des comportements anormaux en termes de consommation énergétique.


In [ ]:
# Détection des outliers par site en utilisant la méthode IQR (Interquartile Range)

print("="*80)
print("DÉTECTION DES OUTLIERS PAR SITE (Méthode IQR)")
print("="*80)

# Calculer Q1, Q3 et IQR pour la consommation moyenne par site
Q1 = site_stats['Consommation moyenne'].quantile(0.25)
Q3 = site_stats['Consommation moyenne'].quantile(0.75)
IQR = Q3 - Q1

# Définir les seuils pour les outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"\n📊 Statistiques pour la détection d'outliers :")
print(f"Q1 (25e percentile) : {Q1:.2f} kWh")
print(f"Q3 (75e percentile) : {Q3:.2f} kWh")
print(f"IQR : {IQR:.2f} kWh")
print(f"Seuil inférieur : {lower_bound:.2f} kWh")
print(f"Seuil supérieur : {upper_bound:.2f} kWh")

# Identifier les sites outliers
site_stats['Est_Outlier'] = (site_stats['Consommation moyenne'] < lower_bound) | (site_stats['Consommation moyenne'] > upper_bound)
outlier_sites = site_stats[site_stats['Est_Outlier']]

print(f"\n🚨 SITES OUTLIERS DÉTECTÉS : {len(outlier_sites)} site(s)")
if len(outlier_sites) > 0:
    print("\n📋 Détails des sites outliers :")
    print(outlier_sites[['Consommation moyenne', 'Consommation médiane', 'Écart-type', 'Nombre de relevés']])

    # Classifier les outliers
    high_outliers = outlier_sites[outlier_sites['Consommation moyenne'] > upper_bound]
    low_outliers = outlier_sites[outlier_sites['Consommation moyenne'] < lower_bound]

    if len(high_outliers) > 0:
        print(f"\n⬆️ Sites avec FORTE consommation anormale : {list(high_outliers.index)}")
    if len(low_outliers) > 0:
        print(f"\n⬇️ Sites avec FAIBLE consommation anormale : {list(low_outliers.index)}")
else:
    print("Aucun site outlier détecté avec la méthode IQR.")

# Visualisation
fig, ax = plt.subplots(figsize=(14, 6))
colors = ['red' if outlier else 'steelblue' for outlier in site_stats['Est_Outlier']]
bars = ax.bar(site_stats.index.astype(str), site_stats['Consommation moyenne'], color=colors, edgecolor='black')

ax.axhline(y=upper_bound, color='red', linestyle='--', linewidth=2, label=f'Seuil supérieur ({upper_bound:.0f} kWh)')
ax.axhline(y=lower_bound, color='orange', linestyle='--', linewidth=2, label=f'Seuil inférieur ({lower_bound:.0f} kWh)')
ax.axhline(y=site_stats['Consommation moyenne'].median(), color='green', linestyle='-', linewidth=2, label=f'Médiane ({site_stats["Consommation moyenne"].median():.0f} kWh)')

ax.set_title('Détection des Sites Outliers - Consommation Moyenne', fontsize=16, fontweight='bold')
ax.set_xlabel('Site ID', fontsize=12)
ax.set_ylabel('Consommation Moyenne (kWh)', fontsize=12)
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


### Analyse de la Consommation par Bâtiment (Building ID)

Examinons maintenant les différences de consommation énergétique entre les **1449 bâtiments** pour identifier les bâtiments avec des comportements énergétiques anormaux.


In [ ]:
# Analyse de la consommation énergétique par bâtiment

print("="*80)
print("ANALYSE DE LA CONSOMMATION ÉNERGÉTIQUE PAR BÂTIMENT")
print("="*80)

# Calculer les statistiques par bâtiment
building_stats = merged_df.groupby('building_id')['meter_reading'].agg([
    ('Nombre de relevés', 'count'),
    ('Consommation moyenne', 'mean'),
    ('Consommation médiane', 'median'),
    ('Consommation totale', 'sum'),
    ('Écart-type', 'std'),
    ('Min', 'min'),
    ('Max', 'max')
]).sort_values('Consommation moyenne', ascending=False)

print(f"\n📊 Nombre total de bâtiments analysés : {len(building_stats)}")
print("\n🔝 TOP 10 des bâtiments avec la plus forte consommation moyenne :")
print(building_stats.head(10))

print("\n🔻 TOP 10 des bâtiments avec la plus faible consommation moyenne :")
print(building_stats.tail(10))

# Statistiques générales
print("\n📈 STATISTIQUES GÉNÉRALES SUR LES BÂTIMENTS :")
print(f"Consommation moyenne globale : {building_stats['Consommation moyenne'].mean():.2f} kWh")
print(f"Consommation médiane globale : {building_stats['Consommation moyenne'].median():.2f} kWh")
print(f"Écart-type global : {building_stats['Consommation moyenne'].std():.2f} kWh")
print(f"Consommation min : {building_stats['Consommation moyenne'].min():.2f} kWh")
print(f"Consommation max : {building_stats['Consommation moyenne'].max():.2f} kWh")
print(f"Ratio max/min : {building_stats['Consommation moyenne'].max() / building_stats['Consommation moyenne'].min():.2f}x")


In [ ]:
# Visualisation de la distribution de la consommation par bâtiment

fig, axes = plt.subplots(2, 2, figsize=(20, 14))

# 1. Histogramme de la consommation moyenne par bâtiment
axes[0, 0].hist(building_stats['Consommation moyenne'], bins=100, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Distribution de la Consommation Moyenne par Bâtiment', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Consommation Moyenne (kWh)', fontsize=12)
axes[0, 0].set_ylabel('Nombre de Bâtiments', fontsize=12)
axes[0, 0].set_yscale('log')
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. Histogramme avec échelle logarithmique sur x
axes[0, 1].hist(np.log1p(building_stats['Consommation moyenne']), bins=100, color='lightcoral', edgecolor='black')
axes[0, 1].set_title('Distribution de log(1 + Consommation Moyenne) par Bâtiment', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('log(1 + Consommation Moyenne)', fontsize=12)
axes[0, 1].set_ylabel('Nombre de Bâtiments', fontsize=12)
axes[0, 1].grid(axis='y', alpha=0.3)

# 3. Top 30 bâtiments consommant le plus
top_30_buildings = building_stats.head(30)
axes[1, 0].barh(range(len(top_30_buildings)), top_30_buildings['Consommation moyenne'], color='steelblue', edgecolor='black')
axes[1, 0].set_yticks(range(len(top_30_buildings)))
axes[1, 0].set_yticklabels([f"Bât. {bid}" for bid in top_30_buildings.index])
axes[1, 0].set_title('TOP 30 - Bâtiments avec la Plus Forte Consommation', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Consommation Moyenne (kWh)', fontsize=12)
axes[1, 0].grid(axis='x', alpha=0.3)
axes[1, 0].invert_yaxis()

# 4. Boxplot de la consommation moyenne par bâtiment
axes[1, 1].boxplot(building_stats['Consommation moyenne'], vert=True)
axes[1, 1].set_title('Boxplot de la Consommation Moyenne par Bâtiment', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('Consommation Moyenne (kWh)', fontsize=12)
axes[1, 1].set_yscale('log')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Détection des outliers par bâtiment - Méthode : Z-Score UNIQUEMENT

from scipy import stats

print("="*80)
print("DÉTECTION DES OUTLIERS PAR BÂTIMENT - MÉTHODE Z-SCORE")
print("="*80)

# Calculer le Z-Score pour chaque bâtiment
building_stats['Z_Score'] = np.abs(stats.zscore(building_stats['Consommation moyenne']))

# Définir un seuil de Z-Score (généralement 3 pour les outliers extrêmes)
z_threshold = 3
building_stats['Est_Outlier_ZScore'] = building_stats['Z_Score'] > z_threshold

outlier_buildings_zscore = building_stats[building_stats['Est_Outlier_ZScore']]

print(f"\n🚨 BÂTIMENTS OUTLIERS DÉTECTÉS (Z-Score > {z_threshold}) : {len(outlier_buildings_zscore)} bâtiment(s) ({len(outlier_buildings_zscore)/len(building_stats)*100:.1f}%)")

if len(outlier_buildings_zscore) > 0:
    print(f"\n📋 TOP 15 des bâtiments outliers (Z-Score) :")
    print(outlier_buildings_zscore.nlargest(15, 'Z_Score')[['Consommation moyenne', 'Z_Score', 'Nombre de relevés']])

# Visualisation Z-Score uniquement
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Distribution des Z-Scores
ax.hist(building_stats['Z_Score'], bins=100, color='coral', edgecolor='black', alpha=0.7)
ax.axvline(x=z_threshold, color='red', linestyle='--', linewidth=2, label=f'Seuil Z-Score = {z_threshold}')
ax.set_title('Distribution des Z-Scores', fontsize=14, fontweight='bold')
ax.set_xlabel('Z-Score (valeur absolue)', fontsize=12)
ax.set_ylabel('Nombre de Bâtiments', fontsize=12)
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


## Caractéristiques Météorologiques vs. Relevés de Compteur

Le dataset contient également plusieurs variables météorologiques qui sont enregistrées au niveau du site, affectant ainsi tous les bâtiments des mêmes sites.


In [ ]:
# Rappel et analyse du dataset météorologique

print("="*80)
print("ANALYSE DU DATASET MÉTÉOROLOGIQUE")
print("="*80)

# Vérifier que weather_df est bien chargé
if 'weather_df' not in globals():
    print("❌ Le dataset météorologique n'est pas chargé. Chargement en cours...")
    dtypes = {
        'site_id': 'int8',
        'air_temperature': 'float32',
        'cloud_coverage': 'float32',
        'dew_temperature': 'float32',
        'precip_depth_1_hr': 'float32',
        'sea_level_pressure': 'float32',
        'wind_direction': 'float32',
        'wind_speed': 'float32'
    }
    weather_file = '../../data/GREENprojet/ashrae/weather_train.csv'
    weather_df = pd.read_csv(weather_file, dtype=dtypes, parse_dates=['timestamp'], low_memory=True)
    print("✅ Dataset météorologique chargé.")

print(f"\n📊 Dimensions du dataset météorologique : {weather_df.shape[0]:,} lignes × {weather_df.shape[1]} colonnes")
print(f"💾 Mémoire utilisée : {weather_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Statistiques descriptives
print("\n📈 Statistiques descriptives des caractéristiques météorologiques :")
weather_stats = weather_df.describe()
print(weather_stats)

# Valeurs manquantes
print("\n❓ Valeurs manquantes par variable :")
missing_weather = weather_df.isnull().sum()
missing_weather_pct = (missing_weather / len(weather_df)) * 100
missing_df_weather = pd.DataFrame({
    'Valeurs manquantes': missing_weather,
    'Pourcentage (%)': missing_weather_pct
})
print(missing_df_weather)

# Période temporelle
print(f"\n📅 Période temporelle :")
print(f"   • Date de début : {weather_df['timestamp'].min()}")
print(f"   • Date de fin : {weather_df['timestamp'].max()}")
print(f"   • Durée : {(weather_df['timestamp'].max() - weather_df['timestamp'].min()).days} jours")
print(f"   • Nombre de sites : {weather_df['site_id'].nunique()}")


In [ ]:
# Visualisation des distributions des variables météorologiques (cela a deja été fait en partie
# plus haut dans le notebook)

weather_features = [
    'air_temperature', 'dew_temperature', 'cloud_coverage',
    'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed'
]

fig, axes = plt.subplots(3, 3, figsize=(20, 16))
axes = axes.flatten()

for i, feature in enumerate(weather_features):
    data = weather_df[feature].dropna()
    axes[i].hist(data, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    axes[i].set_title(f'Distribution de {feature}', fontsize=12, fontweight='bold')
    axes[i].set_xlabel(feature, fontsize=10)
    axes[i].set_ylabel('Fréquence', fontsize=10)
    axes[i].grid(axis='y', alpha=0.3)

    # Ajouter des statistiques sur le graphique
    mean_val = data.mean()
    median_val = data.median()
    axes[i].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Moyenne: {mean_val:.1f}')
    axes[i].axvline(median_val, color='green', linestyle='--', linewidth=2, label=f'Médiane: {median_val:.1f}')
    axes[i].legend(fontsize=8)

# Supprimer les axes inutilisés
for j in range(len(weather_features), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

print("\n✅ Les 7 caractéristiques météorologiques ont été visualisées.")


# Preparing the dataset / Feature Engineering
Reducing the memory size